In [110]:
from modules import Path 

import string
import re
import io
import os
os.environ['TIKA_SERVER_JAR'] = 'https://repo1.maven.org/maven2/org/apache/tika/tika-server/1.19/tika-server-1.19.jar'
from tika import parser

# In dieser Datei soll eine Neue Datei geschrieben.
# Für jeden Jahresabschluss, den diese Datei einließt, soll eine Excel-Arbeitsmappe erstellt 
# werden, die anhand einer KeyWordListe matches abgleicht und den Satz des Matches extrahiert. 

#=============================================================================================
# 0. VORBEREITUNG: Pfade sammlen, KWL laden.
#=============================================================================================

# Pfadliste
pfade = Path.fetch("test_data")
for pfad in pfade:
    print(pfad)

# KWL laden:
KWL_raw = []
with open("KWL.txt", "r", encoding="utf-8") as KWL_file:
    for line in KWL_file:
        KWL_raw.append(line.strip())
len(KWL_raw)


C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Bayer AG2009_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\BMW AG2008_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Deutsche Post AG2010_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\E.ON AG2011_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Evonik Industries AG2011_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Fraport AG FFM Airport2012_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Henkel AG_Co.KGAA2013_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Linde AG_SR_2014.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\MAN SE_SR_2015.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\RWE AG_SR_2016.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\t

68

In [111]:
# wegen etwaiger Dopplungen in der KWL-raw wird diese überarbeitet. 
# Das ist nur für künftige Versionen. Es liegen aktuell (KWL v0.2) keine Dopplungen mehr vor. 

KWL_raw_low = [key.lower() for key in KWL_raw] 
KWL_set = set()
KWL = []

for key in KWL_raw_low:
    KWL_set.add(key)
for key in KWL_set:
    KWL.append(key)
len(KWL)

68

In [112]:
KWL

['korrektur',
 'rückwirkend',
 'umformuliert',
 'ausbesserung',
 'aufdatieren',
 'neufassung',
 'umformulierung',
 'überholt',
 'upgedated',
 'vorjahreswert',
 'neu ermittelt',
 'revision',
 'neuermittlung',
 'abgeändert',
 'bereinigen',
 'nachprüfung',
 'bearbeitung',
 'neuen berechnungsmethoden',
 'angepasst',
 'neuen methoden',
 'berichtigung',
 'neuer methode',
 'überarbeiten',
 'anpassung',
 'updaten',
 'änderung',
 'nicht vergleichbar',
 'ausbessern',
 'update',
 'neue methode',
 'richtigstellung',
 'neu formuliert',
 'neuer berechnungsmethode',
 'aktualisieren',
 'neuformulierung',
 'scope-erweiterung',
 'berichtigt',
 'aktualisiert',
 'aktualisierung',
 'beheben',
 'redigieren',
 'neue berechnungsmethode',
 'neu formulieren',
 'korrigieren',
 'korrigiert',
 'neue methoden',
 'geändert',
 'bereinigt',
 'revidiert',
 'revidieren',
 'neue berechnungsmethoden',
 'nachbesserung',
 'abänderung',
 'korrektion',
 'ausgebessert',
 'fehlerverbesserung',
 'überarbeitet',
 'richtig stellen

In [113]:
# Die vorgegebene KWL ist unvollständig und trotzdem überladen ("Korrektion"?). Ich habe eine eigene Kombination aus KWLs gerbastelt. Dient nur einem Probelauf. 
# Dabei sind regex-Filter als solche für Python, nicht für PERL geschrieben. Bei Bedarf ändere ich das nochmal Jetzt will ich erstmal eine Stark reduzierte Ausgabe erzielen. 

my_KWL_prime = [r'neubehandlung', r'wert\w*', r'daten', r'\w*wechsel', r'technisch\w*\s*gründe\w*', r'korrektur', r'anpassung', r'method\w*', r'aktualisierung', r'update']
my_KWL_sub1 = [r'vorjahr\w*', r'[0-9]{4}']
my_KWL_sub2 = [r'angepasst\w*', r'anpass\w*', r'korrigier\w*', r'aktualisier\w*', r'verbess\w*', r'\w*änder\w*', r'ausbesser\w*', r'aufdatier\w*', r'upgedated', r'geupdated']

my_KWL = [r'neubehandlung', r'wert\w*', r'daten', r'\w*wechsel', r'technisch\w*\s*gründe\w*', r'korrektur', r'anpassung', r'method\w*', r'aktualisierung', r'update', r'vorjahr\w*', r'[0-9]{4}', r'angepasst\w*', r'anpass\w*', r'korrigier\w*', r'aktualisier\w*', r'verbess\w*', r'\w*änder\w*', r'ausbesser\w*', r'aufdatier\w*', r'upgedated', r'geupdated']

In [114]:
counter = 0 
satz = 'Das hier ist ein Beispiel-restatement welches die wert wert wert von 2019 und 2020 geupdeted zur Verfügung stellen soll und dabei wegen eines Wechsels der Methode auf eine Änderung der Daten hinweist.'.lower()
for key in my_KWL:
    if re.search(key, satz):
        counter += len(re.findall(key, satz))
print(counter)

9


In [115]:
satz

'das hier ist ein beispiel-restatement welches die wert wert wert von 2019 und 2020 geupdeted zur verfügung stellen soll und dabei wegen eines wechsels der methode auf eine änderung der daten hinweist.'

In [116]:
#=============================================================================================
# 1. MATCH COUNT
#=============================================================================================

# Pfadliste
pfade = Path.fetch("test_data")

In [117]:
for file in pfade:
    print(file)
    
    # Jahresabschluss einlesen
    parsedPDF = parser.from_file(file) 

    # Name des Dokuments speichern
    name = os.path.basename(file)[0:-4] #letzter Block um ".pdf" abzuschneiden
    
    # Jahr des Berichts speichern
    year = int(re.findall(r'[0-9]{4}', file)[0])
    
    # Einzelne Seiten aufrufen und Berichtslänge speichern
    pages_raw = parsedPDF['content'].split('\n\n\n\n')
    pages = [page for page in pages_raw if len(page)] # HIER SIND NOCH UMLAUTE, bEIM SPEICHERN NACHHER NICHT?
    length = len(pages_raw)
    
    # Seitenzähler und satz_dict innitiieren
    page_number = 0
    satz_dict = {}
    
    for page in pages:
        
        page_number += 1
        
        # Text_prep: Absätze (\n), Tabs (\t) und Semikoli raus; alles klein schreiben ; Liste aus Sätzen erstellen 
        text = page.replace('\n', "").replace('\t', "").replace(";", '').strip().lower()
        satz_liste = text.split('.')

        # Satzweise die gesamte KWL checken lassen. Wenn Match, dann Satz m it Nummer in Match_Liste ablegen
        satz_counter = 0
        for satz in satz_liste:
            
            satz_counter += 1
            
            kw_counter = 0 
        
            for key in my_KWL_prime:
                if re.search(key, satz):
                    kw_counter += len(re.findall(key, satz))
            for key in my_KWL_sub1:
                if re.search(key, satz):
                    kw_counter += len(re.findall(key, satz))
            for key in my_KWL_sub2:
                if re.search(key, satz):
                    kw_counter += len(re.findall(key, satz))
            
            print(kw_counter)
            satz_dict(satz) = kw_counter
    
    satz_dict = dict(sorted(satz_dict.items(), key=lambda item: item[1], reverse = True)) # hier hab ich ein absteigend sortiertes satz_dict alá (satz: anzahl_matches),...
    satz_dict_rev = [(count, satz) for (satz, count) in list(satz_dict.items())]
    
    with open("./arbeitsprobe_perl.csv", "a", encoding="utf-8") as File:
        
        for i in range(len(satz_dict)):
            
            if satz_dict_rev[1] >= 2: # Annahme: Man braucht 2 Matches um ein restatement auszumachen. 
                File.write(u"{};{};{};{};{}\n".format(name, year, length, satz)) #page_number, satz_counter, key,

SyntaxError: cannot assign to function call (<ipython-input-117-08b19e517792>, line 49)

In [ ]:
for file in pfade:
    print(file)
    
    # Jahresabschluss einlesen
    parsedPDF = parser.from_file(file) 

    # Name des Dokuments speichern
    name = os.path.basename(file)[0:-4] #letzter Block um ".pdf" abzuschneiden
    
    # Jahr des Berichts speichern
    year = int(re.findall(r'[0-9]{4}', file)[0])
    
    # Einzelne Seiten aufrufen und Berichtslänge speichern
    pages_raw = parsedPDF['content'].split('\n\n\n\n')
    pages = [page for page in pages_raw if len(page)] # HIER SIND NOCH UMLAUTE, bEIM SPEICHERN NACHHER NICHT?
    length = len(pages_raw)
    
    # Seitenzähler und satz_dict innitiieren
    page_number = 0
    satz_dict = {}
    
    for page in pages:
        
        page_number += 1
        
        # Text_prep: Absätze (\n), Tabs (\t) und Semikoli raus; alles klein schreiben ; Liste aus Sätzen erstellen 
        text = page.replace('\n', "").replace('\t', "").replace(";", '').strip().lower()
        satz_liste = text.split('.')

        # Satzweise die gesamte KWL checken lassen. Wenn Match, dann Satz m it Nummer in Match_Liste ablegen
        satz_counter = 0
        for satz in satz_liste:
            
            satz_counter += 1
            
            kw_counter = 0 
        
            for key in my_KWL_prime:
                if re.search(key, satz):
                    kw_counter += len(re.findall(key, satz))
            for key in my_KWL_sub1:
                if re.search(key, satz):
                    kw_counter += len(re.findall(key, satz))
            for key in my_KWL_sub2:
                if re.search(key, satz):
                    kw_counter += len(re.findall(key, satz))
            
            print(kw_counter)

C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Bayer AG2009_SR.pdf


2021-07-10 17:24:16,845 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to C:\Users\test\AppData\Local\Temp\tika-server.jar.
